In [ ]:
##This code is for finding minimal value for training data by convex formulation based on the idea from the paper
##"Neural Networks are Convex Regularizers: Exact Polynomial-time Convex Optimization Formulations for Two-layer Networks"
##regression (squared loss), fixed design
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import torch

In [ ]:
def relu(x):
    return np.maximum(0,x)
def drelu(x):
    return x>=0
n=5 # number of samples
d=2 # dimension of the data
beta=1e-3 # regularization parameter

#Adjust variables for each experiment
dv=5.0 #standard deviation of noise

In [ ]:
# Input data
X = torch.tensor([[1.0, -2.0],
                  [1.0, -1.0],
                  [1.0, 0.0],
                  [1.0, 1.0],
                  [1.0, 2.0]], dtype=torch.float32)

Y = torch.tensor([1.0, 1.0, 1.0, -1.0, -1.0], dtype=torch.float32)
# Fix the random seed for reproducibility
torch.manual_seed(42)
# Generate Gaussian noise with the same shape as Y (mean=0, std=dv)
noise = dv*torch.randn_like(Y)

# Add Gaussian noise to Y
Y = Y + noise

In [ ]:
# Check training data
print(X)
print(Y)
Y=Y.numpy()

tensor([[ 1., -2.],
        [ 1., -1.],
        [ 1.,  0.],
        [ 1.,  1.],
        [ 1.,  2.]])
tensor([ 2.6835,  1.6440,  2.1723,  0.1517, -6.6143])


In [ ]:
# Matrix to store hyperplane arrangements for X
dmat=np.empty((n,0))

In [ ]:
# Finite approximation of all possible sign patterns
for i in range(int(1e5)):
    u=np.random.randn(d,1)
    # u is a 2D array of shape (d, 1) filled with random floats sampled from a standard normal distribution (mean = 0, standard deviation = 1)
    dmat=np.append(dmat,drelu(np.dot(X,u)),axis=1)
    # The function appends the result of drelu(np.dot(X, u)) as a new column to the right of the dmat array.

dmat=(np.unique(dmat,axis=1))

<ipython-input-6-286a78cbd0b5>:5: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  dmat=np.append(dmat,drelu(np.dot(X,u)),axis=1)


In [ ]:
# For this input X, the number of hyperplane arrangements are 10
# Ensure that dmat.shape is (5,10) for this data
# If not, increase the range for i
print(dmat.shape)

(5, 10)


In [ ]:
# Optimal CVX
m1=dmat.shape[1] #number of columns in dmat
# Uopt1 and Uopt2 to represent parameters to be optimised
Uopt1=cp.Variable((d,m1))
Uopt2=cp.Variable((d,m1))

In [ ]:
# yopt1 and yopt2 to represent a fixed vector DXv and DXt
yopt1=cp.Parameter((n,1))
yopt2=cp.Parameter((n,1))
yopt1=cp.sum(cp.multiply(dmat,(X*Uopt1)),axis=1)
yopt2=cp.sum(cp.multiply(dmat,(X*Uopt2)),axis=1)

/usr/local/lib/python3.11/dist-packages/cvxpy/expressions/expression.py:674: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cvxpy/expressions/expression.py:674: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.warn(msg, UserWarning)


In [ ]:
# We use MSE+regularization term as a performance metric
cost=cp.sum((Y - (yopt1 - yopt2))**2)/n + (beta/2)*(cp.norm(Uopt1, 'fro')**2 + cp.norm(Uopt2, 'fro')**2)

In [ ]:
# Add constraints
constraints=[]
constraints+=[cp.multiply((2*dmat-np.ones((n,m1))),(X*Uopt1))>=0]
constraints+=[cp.multiply((2*dmat-np.ones((n,m1))),(X*Uopt2))>=0]

/usr/local/lib/python3.11/dist-packages/cvxpy/expressions/expression.py:674: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 3 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cvxpy/expressions/expression.py:674: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 4 times so far.

  warnings.warn(msg, UserWarning)


In [ ]:
# Solve convex program
prob=cp.Problem(cp.Minimize(cost),constraints)
prob.solve()
cvx_opt=prob.value

In [ ]:
print("Convex program objective value: ",cvx_opt)

Convex program objective value:  0.012772684984779462
